### Towards Data Science
#### Build and deploy your own AI chatbot to hundreds of users
https://towardsdatascience.com/build-your-own-ai-chatbot-84d3b08e86f3

### CollectedNotes.com
#### Create a Stateful Chat AI in 10 minutes 
https://collectednotes.com/chai/create-a-stateful-chat-ai-in-10-minutes


In [1]:
#!pip install npu
#npu in c:\users\hilton.netta\anaconda3\lib\site-packages (0.3.900)
#!pip install retry
#retry in c:\users\hilton.netta\anaconda3\lib\site-packages

To make things easier I put all the relevant code together in this GitHub repo: “Example Chai Bot”: https://github.com/tr416/example_chai_bot

First we need a way to interact with GPT-J, our AI model. This is done via the FineTunedAPI class below:

In [5]:
import npu
import retry

class FineTunedAPI(object):
  def __init__(self, temperature, rep_penalty):
    API_TOKEN = "<EMAIL dev@chai.ml for token>"
    npu.api(API_TOKEN, deployed=True)
    self.temperature = temperature
    self.rep_penalty = rep_penalty

  @retry.retry(tries=3, backoff=2)
  def request(self, data):
    """
    input: data is a string, the message we want a response to.
    output: (string) reponse from the model.
    """
    data = data[-2048:]
    kwargs = {
    "remove_input": True,  # whether to return your input
    "do_sample": True,  # important to get realistic sentences and not just MLE
    "temperature": self.temperature,
    "response_length": 32,  # how many response tokens to generate
    "repetition_penalty": self.rep_penalty,  # 1 is the default
    "eos_token_id": 198,  # this is \n
    }
    model_id = "<EMAIL dev@chai.ml for model_id>"
    output = npu.predict(model_id, [data], kwargs)
    resp = output[0]["generated_text"]
    return resp

ModuleNotFoundError: No module named 'npu'

Notice how simple it is: there is just a “request” function which we have to give a string to (this is the message we want a response to). In this API we set the repetition penalty, temperature, and response length. Just as in the playground example earlier.<br><br>
Great, now that we can query GPT-J, we just need to take that, and turn it into a chatbot. We’re going to want to give the bot a prompt, and an example conversation. Just like in the playground. The ChatAI class below does this for us in the __init__ function.

In [8]:
import re

class ChatAI(object):                                                           
    def __init__(self, model):                                                  
        self.bot_name = "Chris"                                                 
        self.chat_history = [                                                   
            "User: hi",                                                         
            "{bot_name}: Hello there! I'm Chris. I work as a coder but I love rock climbing. What about you?",
            "User: hi chris, nice to meet you",                                 
            "{bot_name}: Nice to meet you! How are you feeling today?",         
        ]                                                                       
        self.max_history = 104                                                  
        self.model = model                                                      
        self.prompt = "Chris is a 30 year old male software developer.  He is also a Christian and likes to give people good advice.  Chris speaks softly and politely. Chris is friendly.  Chris wants to be User's friend."
                                          
                                                                                
    def get_resp(self, input_message):                                          
        """"                                                                    
        When we send a request the chat_history has to look like below for      
        the model to know it is the bot speaking:                               
        User: input_message                                                     
        {bot_name}:                                                             
        """                                                                     
        self._update_chat_history(message=input_message, sender="User")         
        self._update_chat_history(message="", sender="{bot_name}")              
                                                                                
        request = self._prepare_request()                                       
        raw_response = self.model.request(request)                              
        formatted_response = self._format_model_response(raw_response)          
                                                                                
        self.chat_history[-1] += formatted_response                             
        return formatted_response    
      
    def _prepare_request(self):                                                                    
        formatted_chat_history = self.chat_history[-self.max_history :]                            
        formatted_chat_history = "\n".join(formatted_chat_history)                                 
        request = self.prompt + formatted_chat_history                                             
        request = request.replace("{bot_name}", self.bot_name)                                     
        return request                                                                             
                                                                                                   
    def _format_model_response(self, resp):                                                        
        partial_chat = re.split(                                                                   
            "User|{lower_bot_name}:|{bot_name}:|\n|\$".format(                  
                lower_bot_name=self.bot_name.lower(), bot_name=self.bot_name    
            ),                                                                  
            resp,                                                               
        )[0]                                                                    
        partial_chat = partial_chat.strip(" ")                                  
        if len(partial_chat) == 0:                                              
            partial_chat = "..."                                                
        return partial_chat                                                     
                                                                                
    def _update_chat_history(self, message, sender):                            
        cleaned_message = message.strip(" ")                                    
        self.chat_history += [f"{sender}: {cleaned_message}"]  

You can safely ignore all the code except from the __init__ where we define the personality of the bot by giving it a prompt and a conversation history.<br><br>
Perfect, now we can package the two classes above together nicely and we will have a chatbot!

In [14]:
#!pip install -U chaipy
#Successfully installed chaipy-0.3.12 halo-0.0.31 log-symbols-0.0.14 segno-1.4.1 spinners-0.0.24 termcolor-1.1.0
#!pip install gpt
#Successfully installed gpt-0.2
#!pip install utils
#Successfully installed utils-1.0.1

In [15]:
from chai_py import ChaiBot, Update
from gpt import ChatAI, FineTunedAPI
from utils import truncate


class Replica(ChaiBot):
    def setup(self):
        self.logger.info("Setting up...")
        self.model = ChatAI(FineTunedAPI(temp=0.5, rep_penalty=1.15))
        self.max_len = 100

    async def on_message(self, update):
        return self.respond(update.latest_message.text)

    def respond(self, message):
        if message == "__first":
            output = "Hi there! I'm Chris, a software developer from London. What about yourself?"
        else:
            output = self.model.get_resp(message)
            if len(output) > self.max_len:
                output = truncate(output, self.max_len)
        return output


ImportError: cannot import name 'ChatAI' from 'gpt' (C:\Users\hilton.netta\Anaconda3\lib\site-packages\gpt\__init__.py)

How to fix the error : "cannot import name 'GPT' from "gpt"
https://stackoverflow.com/questions/68444704/how-to-fix-the-error-cannot-import-name-gpt-from-gpt